## Import Library and Data Input

In [ ]:
!pip install -qqq pyomo
# !apt-get install -y -qq glpk-utils
# !apt-get install -y -qq coinor-cbc
# !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
# !unzip -o -q ipopt-linux64
!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
!unzip -o -q bonmin-linux64
# !wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
# !unzip -o -q couenne-linux64

import math
import numpy as np
import pandas as pd
from pyomo.environ import *
from itertools import product
from tqdm import tqdm

     |████████████████████████████████| 9.1 MB 7.1 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 


In [ ]:
#@title <b>Download data file from Onedrive</b>
#@markdown ---

input_url = 'https://glowgroup-my.sharepoint.com/:x:/g/personal/paroon_k_gpscgroup_com/EU3TMbclhmROv3X6pK9-I24B8LZrKV1aAIYa9AdffW4M3Q?e=uBKiFa' #@param {type:'string'}
output_dir = "coal_spec.xlsx"
split_url = input_url.rfind('?')
converted_url = input_url[:split_url] + '?download=1'
!wget -q -O "$output_dir" "$converted_url"

# %load_ext google.colab.data_table
pd.options.display.float_format = "{:,.2f}".format
pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False

coal_df = pd.read_excel(output_dir, sheet_name='Coal_Data')
coal_df.set_index('Supplier', inplace=True)
coal_df = coal_df.astype('float')
print('coal_df:')
print(coal_df)

op_df = pd.read_excel(output_dir, sheet_name='Operation')
op_df.set_index('Date', inplace=True)
op_df.dropna(inplace=True)
print('\nop_df:')
print(op_df)

shipment_df = pd.read_excel(output_dir, sheet_name='Shipment')
shipment_df.set_index('Date', inplace=True)
shipment_df.dropna(inplace=True)
print('\nshipment_df:')
print(shipment_df)

limit_cfb12_df = pd.read_excel(output_dir, sheet_name='Limit_CFB12')
limit_cfb12_df.dropna(inplace=True)
print('\nlimit_cfb12_df:')
print(limit_cfb12_df)

limit_cfb3_df = pd.read_excel(output_dir, sheet_name='Limit_CFB3')
limit_cfb3_df.dropna(inplace=True)
print('\nlimit_cfb3_df:')
print(limit_cfb3_df)

coal_df:
          Total Moisture (%)  Inherent Moisture (%)  Ash Content (%)  Volatile Matter (%)  Fixed Carbon (%)  Sulphur Content (%)  Gross Calorific Value (kcal/kg)  Carbon  Hydrogen  Nitrogen  Oxygen  SiO2  Al2O3  Fe2O3   CaO  MgO  Na2O  K2O  TiO2  Mn3O4   SO3  P2O5  %S dry  B/A  Slacking Index  Fouling Index  Coal Cost (THB/ton)  Coal Remaining (ton)
Supplier                                                                                                                                                                                                                                                                                                                                                       
Banpu                  29.05                  11.38             3.74                34.87             32.34                 0.36                         4,758.37   49.27      3.64      0.71   13.24 45.42  14.18  16.40  7.43 5.65  0.57 1.24  0.56   0.24  7.49  0.16    0.50 0.52          

In [ ]:
#@title <b>← Simulation Config</b>
#@markdown ---

# _______Function______
def shift_list(l, idx, shift_idx):
    return l[l.index(idx) + shift_idx]

def norm(x, min, max):
    return (x-min)/(max-min)

def sigmoid(x, a, b, c, d):
    return a / (b + math.e**(-c*(x-d)))

# __Simulation_Config___
Start_Date = '2019-12-31' #@param {type:'date'}
Sim_Week_Total =  52#@param {type:'integer'}
Sim_Week_Duration =  4
Sim_Week_Inc_Step =  2

# supplier = coal_df.index.tolist()[:5]
supplier = coal_df.index.tolist()

# ______Constants_______
ratio_step = 10
n_ratio_step = 100/ratio_step

Qro12 = 286
Qro12_norm = norm(Qro12, min=200, max=330)
Qro3 = 282
Qro3_norm = norm(Qro3, min=200, max=330)
T_stack = norm(165, min=120, max=185)
Excess_O2 = norm(0.025, min=0, max=0.1)
Pa_ratio = 0.64

param_list = ['GCV', 'TM', '%H', '%S', '%Ash', '%Fe2O3', '%Na2O']
param_lookup_list = ['Gross Calorific Value (kcal/kg)', 'Total Moisture (%)', 'Hydrogen', 'Sulphur Content (%)', 'Ash Content (%)', 'Fe2O3', 'Na2O']
param_min = [17000, 0, 0 , 0, 0]
param_max = [29000, 36, 10 , 1, 10]

lime_price = 690
flyash_price = 250
bottomash_price = 757.95

# ______Dataframes_______
df = pd.DataFrame(pd.date_range(Start_Date, periods=(Sim_Week_Total * 7) + 1, freq='D'), columns=['Date']).set_index('Date')
op_df = op_df.loc[[d for d in op_df.index if d in df.index], :]
shipment_df = shipment_df.loc[[d for d in shipment_df.index if d in df.index], :]

df.loc[df.index[0]:, [f"OP_CFB{n}" for n in [1, 2, 3]]] = 0
df.loc[[d for d in op_df.index], [f"OP_CFB{i + 1}" for i in range(3)]] = op_df[[f"CFB{i + 1}" for i in range(3)]].values/24

df.loc[df.index[0]:, [f"In_{sp}" for sp in supplier]] = 0
df.loc[[d for d in shipment_df.index], [f"In_{sp}" for sp in supplier]] = shipment_df[[f"{sp}" for sp in supplier]].values

df[[f"Remain_{sp}" for sp in supplier]] = 0
df.loc[Start_Date, [f"Remain_{sp}" for sp in supplier]] = coal_df.loc[supplier, 'Coal Remaining (ton)'].tolist()

# save df_daily for later use
df_daily = df.copy()
# print(df_daily)

# Resample to week
df = df.reset_index().resample(f"W-{df.index[0].day_name()[:3]}", on='Date').sum()
df

,OP_CFB1,OP_CFB2,OP_CFB3,In_Banpu,In_Logplus,In_Mac,In_Tiger,In_AVRA,In_Spot1,In_Spot2,In_Spot3,Remain_Banpu,Remain_Logplus,Remain_Mac,Remain_Tiger,Remain_AVRA,Remain_Spot1,Remain_Spot2,Remain_Spot3
Date,,,,,,,,,,,,,,,,,,,
2019-12-31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"69,336.07",0.00,"19,000.00","1,602.00",0.00,0.00,0.00
2020-01-07,6.88,6.96,6.94,0.00,0.00,0.00,"33,862.12",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-01-14,6.90,6.95,6.90,"54,004.86",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-01-21,7.03,6.91,6.60,0.00,"55,050.26",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-01-28,6.99,6.92,6.89,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-02-04,6.51,6.61,6.90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-02-11,6.71,6.94,6.88,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-02-18,6.77,6.95,6.47,0.00,0.00,"18,602.80",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2020-02-25,6.96,5.57,6.57,0.00,"55,058.80",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Code

In [ ]:
def simulation(df, show_solver_log=False):
    # ___Initialize model___
    m = ConcreteModel()

    # ______Constants_______
    date_rng = df.index.tolist()

    m.zero = Param(default=0)

    # _______Variables______
    m.remain = Var(date_rng, supplier, domain=NonNegativeReals)
    m.remain_notzero = Var(date_rng[1:], supplier, domain=Binary)

    m.cfb12_ratio = Var(date_rng[1:], supplier, domain=NonNegativeReals, bounds=(0, n_ratio_step))
    m.cfb3_ratio = Var(date_rng[1:], supplier, domain=NonNegativeReals, bounds=(0, n_ratio_step))
    m.cfb12_select = Var(date_rng[1:], supplier, domain=Binary)
    m.cfb3_select = Var(date_rng[1:], supplier, domain=Binary)
    m.cfb12_use = Var(date_rng[1:], supplier, domain=NonNegativeReals)
    m.cfb3_use = Var(date_rng[1:], supplier, domain=NonNegativeReals)

    # ______Equations_______
    m.cons = ConstraintList()

    # total ratio = 100%
    for d in date_rng[1:]:
        m.cons.add(sum(m.cfb12_ratio[d, s] for s in supplier) == n_ratio_step)
        m.cons.add(sum(m.cfb3_ratio[d, s] for s in supplier) == n_ratio_step)

    # if ratio > 0 then select = 1, else if ratio == 0 then select = 0
    for d, s in product(date_rng[1:], supplier):
        m.cons.add(m.cfb12_ratio[d, s] <= n_ratio_step * m.cfb12_select[d, s])
        m.cons.add(m.cfb12_ratio[d, s] >= m.cfb12_select[d, s])
        m.cons.add(m.cfb3_ratio[d, s] <= n_ratio_step * m.cfb3_select[d, s])
        m.cons.add(m.cfb3_ratio[d, s] >= m.cfb3_select[d, s])

    # total number of select not exceed 2
    for d in date_rng[1:]:
        m.cons.add(sum(m.cfb12_select[d, s] for s in supplier) <= 2)
        m.cons.add(sum(m.cfb3_select[d, s] for s in supplier) <= 2)

    # calculate coal mixing parameters
    cfb12_mix = {p: {d: sum(m.cfb12_ratio[d, s] * coal_df.loc[s, param_lookup_list[param_list.index(p)]] for s in supplier) 
                        / sum(m.cfb12_ratio[d, s] for s in supplier) for d in date_rng[1:]} for p in param_list}
    cfb3_mix = {p: {d: sum(m.cfb3_ratio[d, s] * coal_df.loc[s, param_lookup_list[param_list.index(p)]] for s in supplier)
                        / sum(m.cfb3_ratio[d, s] for s in supplier) for d in date_rng[1:]} for p in param_list}
    
    cfb12_mix_norm = {p: {d: norm(cfb12_mix[p][d]*(4.1868 if p == 'GCV' else 1), param_min[param_list.index(p)], param_max[param_list.index(p)]) for d in date_rng[1:]} for p in param_list[:5]}
    cfb3_mix_norm = {p: {d: norm(cfb3_mix[p][d]*(4.1868 if p == 'GCV' else 1), param_min[param_list.index(p)], param_max[param_list.index(p)]) for d in date_rng[1:]} for p in param_list[:5]}
    
    # control mixing parameters in boundary
    for d in date_rng[1:]:
        for p in limit_cfb12_df['Parameter'].tolist():
            x = limit_cfb12_df.loc[limit_cfb12_df['Parameter'] == p]
            m.cons.add((x['Lower Bound'].values[0], cfb12_mix[p][d], x['Upper Bound'].values[0]))
        for p in limit_cfb3_df['Parameter'].tolist():
            x = limit_cfb3_df.loc[limit_cfb3_df['Parameter'] == p]
            m.cons.add((x['Lower Bound'].values[0], cfb3_mix[p][d], x['Upper Bound'].values[0]))
    
    #Correction Tstack when Fe and Na over limit 
    T_Fe12 = {d: (sigmoid(cfb12_mix['%Fe2O3'][d], 5, 1, 40, 11.5) + sigmoid(cfb12_mix['%Fe2O3'][d], 10, 1, 40, 14.5)) for d in date_rng[1:]}
    T_Fe3 = {d: (sigmoid(cfb3_mix['%Fe2O3'][d], 5, 1, 40, 11.5) + sigmoid(cfb3_mix['%Fe2O3'][d], 10, 1, 40, 14.5)) for d in date_rng[1:]}

    T_Na12 = {d: (sigmoid(cfb12_mix['%Na2O'][d], 5, 1, 40, 2.25) + sigmoid(cfb12_mix['%Na2O'][d], 10, 1, 40, 5)) for d in date_rng[1:]}
    T_Na3 = {d: (sigmoid(cfb3_mix['%Na2O'][d], 5, 1, 200, 2.25) + sigmoid(cfb3_mix['%Na2O'][d], 10, 1, 40, 5)) for d in date_rng[1:]}
    
    T12 = {d: norm(165 + T_Fe12[d] + T_Na12[d], min=120, max=185) for d in date_rng[1:]}
    T3 = {d: norm(165 + T_Fe3[d] + T_Na3[d], min=120, max=185) for d in date_rng[1:]}

    # calculate lime flow, boiler eff and coal consumption
    cfb12_lime_flow = {d: 1.098044 - 0.30009*Qro12_norm - 2.205101*cfb12_mix_norm['%S'][d] + 4.485632*(cfb12_mix_norm['%S'][d]**2) for d in date_rng[1:]}
    cfb12_lime_flow_norm = {d: norm(cfb12_lime_flow[d], min=0, max=3.5) for d in date_rng[1:]}
    cfb12_ash_flow = {d: -0.157 + 0.1464*Qro12_norm - 0.1778*cfb12_mix_norm['GCV'][d] + 0.7089*cfb12_lime_flow_norm[d] + 0.423*cfb12_mix_norm['%Ash'][d]
                    - 0.056*Excess_O2 + 0.1962*Pa_ratio for d in date_rng[1:]}
    cfb12_boiler_eff = {d: 98.2894 + 0.2862*Qro12_norm + 5.453*cfb12_mix_norm['GCV'][d] - 4.5807*cfb12_mix_norm['TM'][d]
                        - 17.9721*cfb12_mix_norm['%H'][d] - 0.4126*cfb12_mix_norm['%Ash'][d] - 2.7522*cfb12_lime_flow_norm[d]
                        - 2.9131*T12[d] - 2.6225*Excess_O2 for d in date_rng[1:]}

    cfb3_lime_flow = {d: 1.098044 - 0.30009*Qro3_norm - 2.205101*cfb3_mix_norm['%S'][d] + 4.485632*(cfb3_mix_norm['%S'][d]**2) for d in date_rng[1:]}
    cfb3_lime_flow_norm = {d: norm(cfb3_lime_flow[d], min=0, max=3.5) for d in date_rng[1:]}
    cfb3_ash_flow = {d: -0.157 + 0.1464*Qro3_norm - 0.1778*cfb3_mix_norm['GCV'][d] + 0.7089*cfb3_lime_flow_norm[d] + 0.423*cfb3_mix_norm['%Ash'][d]
                    - 0.056*Excess_O2 + 0.1962*Pa_ratio for d in date_rng[1:]}
    cfb3_boiler_eff = {d: 98.2894 + 0.2862*Qro3_norm + 5.453*cfb3_mix_norm['GCV'][d] - 4.5807*cfb3_mix_norm['TM'][d]
                        - 17.9721*cfb3_mix_norm['%H'][d] - 0.4126*cfb3_mix_norm['%Ash'][d] - 2.7522*cfb3_lime_flow_norm[d]
                        - 2.9131*T3[d] - 2.6225*Excess_O2 for d in date_rng[1:]}

    # calculate coal use
    cfb12_coal_flow = {}
    cfb3_coal_flow = {}
    cfb12_use_sum = {}
    cfb3_use_sum = {}
    for d in date_rng[1:]:
        cfb12_coal_flow[d] = (Qro12*1000) / (cfb12_boiler_eff[d]/100) / (cfb12_mix['GCV'][d]*4.1868)
        cfb3_coal_flow[d] = (Qro3*1000) / (cfb3_boiler_eff[d]/100)  /(cfb3_mix['GCV'][d]*4.1868)

        cfb12_use_sum[d] = cfb12_coal_flow[d] * df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum() * (3600*24) / 1000
        cfb3_use_sum[d] = cfb3_coal_flow[d] * df.loc[d, 'OP_CFB3'] * (3600*24) / 1000

    for d, s in product(date_rng[1:], supplier):
        m.cons.add(m.cfb12_use[d, s] == (m.cfb12_ratio[d, s] / n_ratio_step) * cfb12_use_sum[d])
        m.cons.add(m.cfb3_use[d, s] == (m.cfb3_ratio[d, s] / n_ratio_step) * cfb3_use_sum[d])

    # calculate coal remain
    for d, s in product(date_rng, supplier):
        if d == date_rng[0]:
            m.cons.add(m.remain[d, s] == df.loc[d, f"Remain_{s}"])
        else:
            m.cons.add(m.remain[d, s] == m.remain[shift_list(date_rng, d, -1), s] - m.cfb12_use[d, s] - m.cfb3_use[d, s] + df.loc[d, f"In_{s}"])

    # coal remain >= 0 (exclude incoming shipment)
    for d, s in product(date_rng[1:], supplier):
        m.cons.add(m.remain[shift_list(date_rng, d, -1), s] - m.cfb12_use[d, s] - m.cfb3_use[d, s] >= 0)

    # if remain > 0 then notzero = 1, else if remain == 0 then notzero = 0
    for d, s in product(date_rng[1:], supplier):
        m.cons.add(m.remain[d, s] <= 999999 * m.remain_notzero[d, s])
        m.cons.add(m.remain[d, s] >= m.remain_notzero[d, s])

    # calculate cost
    coal_cost = {d: sum((m.cfb12_use[d, s] + m.cfb3_use[d, s]) * coal_df.loc[s, 'Coal Cost (THB/ton)'] for s in supplier) for d in date_rng[1:]}
    lime_cost = {d: (cfb12_lime_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum() + cfb3_lime_flow[d]*df.loc[d, 'OP_CFB3']) * (3600*24) / 1000 * lime_price for d in date_rng[1:]}
    flyash_cost = {d: (cfb12_ash_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum()*0.9 + cfb3_ash_flow[d]*df.loc[d, 'OP_CFB3']*0.9) * (3600*24) / 1000 * flyash_price  for d in date_rng[1:]}
    bottomash_cost = {d: (cfb12_ash_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum()*0.1 + cfb3_ash_flow[d]*df.loc[d, 'OP_CFB3']*0.1) * (3600*24) / 1000 * bottomash_price  for d in date_rng[1:]}
    fe2o3_cost = {d: cfb12_use_sum[d] * (sigmoid(cfb12_mix['%Fe2O3'][d], 355.34, 1, 40, 14.5)) + 
                cfb3_use_sum[d] * (sigmoid(cfb3_mix['%Fe2O3'][d], 355.34, 1, 40, 14.5)) for d in date_rng[1:]}
    na2o_cost = {d: cfb12_use_sum[d] * (sigmoid(cfb12_mix['%Na2O'][d], 355.34, 1, 40, 5)) + 
                cfb3_use_sum[d] * (sigmoid(cfb3_mix['%Na2O'][d], 355.34, 1, 40, 5)) for d in date_rng[1:]}
    remain_penalty = {d: sigmoid(sum(m.remain_notzero[d, s] for s in supplier), 1e8, 1, 100, 3.5) for d in date_rng[1:]}

    # ______Objective_______
    # m.obj = Objective(expr=sum(coal_cost[d] + lime_cost[d] + flyash_cost[d] + bottomash_cost[d] + fe2o3_cost[d] + na2o_cost[d] + remain_penalty[d] for d in date_rng[1:]), sense=minimize)
    m.obj = Objective(expr=sum(lime_cost[d] + flyash_cost[d] + bottomash_cost[d] + fe2o3_cost[d] + na2o_cost[d] + remain_penalty[d] for d in date_rng[1:]), sense=minimize)

    #_____Solve Problem_____
    # solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
    # solver = SolverFactory('cbc', executable='/usr/bin/cbc')
    # solver = SolverFactory('ipopt', executable='/content/ipopt')
    solver = SolverFactory('bonmin', executable='/content/bonmin')
    # solver = SolverFactory('couenne', executable='/content/couenne')

    result = solver.solve(m, tee=show_solver_log)

    #_______Results________
    result_df = pd.DataFrame(index=df.index)

    for d in reversed(date_rng):
        if d == date_rng[0]:
            result_df.loc[d, df.columns.tolist()] = df.loc[d, :]

        else:
            result_df.loc[d, 'Cost_Coal'] = coal_cost[d]()
            result_df.loc[d, 'Cost_Lime'] = lime_cost[d]()
            result_df.loc[d, 'Cost_FlyAsh'] = flyash_cost[d]()
            result_df.loc[d, 'Cost_BottomAsh'] = bottomash_cost[d]()
            result_df.loc[d, 'Cost_Fe2O3'] = fe2o3_cost[d]()
            result_df.loc[d, 'Cost_Na2O'] = na2o_cost[d]()
            result_df.loc[d, 'Cost_Total'] = result_df.loc[d, [f"Cost_{x}" for x in ['Coal', 'Lime', 'FlyAsh', 'BottomAsh', 'Fe2O3', 'Na2O']]].sum()
            result_df.loc[d, 'Penalty_Remain'] = remain_penalty[d]()

            result_df.loc[d, [f"Use_{s}" for s in supplier]] = [m.cfb12_use[d, s]() + m.cfb3_use[d, s]() for s in supplier]
            result_df.loc[d, 'Use_Total'] = result_df.loc[d, [f"Use_{s}" for s in supplier]].sum()

            result_df.loc[d, [f"In_{s}" for s in supplier]] = df.loc[d, [f"In_{s}" for s in supplier]]
            result_df.loc[d, 'In_Total'] = result_df.loc[d, [f"In_{s}" for s in supplier]].sum()

            result_df.loc[d, [f"Remain_{s}" for s in supplier]] = [m.remain[d, s]() for s in supplier]
            result_df.loc[d, 'Remain_Total'] = result_df.loc[d, [f"Remain_{s}" for s in supplier]].sum()

            result_df.loc[d, [f"NotZero_{s}" for s in supplier]] = [m.remain_notzero[d, s]() for s in supplier]
            result_df.loc[d, 'NotZero_Total'] = result_df.loc[d, [f"NotZero_{s}" for s in supplier]].sum()

            result_df.loc[d, [f"CFB12_Ratio_{s}" for s in supplier]] = [m.cfb12_ratio[d, s]() for s in supplier]
            result_df.loc[d, 'CFB12_Ratio_Total'] = result_df.loc[d, [f"CFB12_Ratio_{s}" for s in supplier]].sum()
            result_df.loc[d, [f"CFB3_Ratio_{s}" for s in supplier]] = [m.cfb3_ratio[d, s]() for s in supplier]
            result_df.loc[d, 'CFB3_Ratio_Total'] = result_df.loc[d, [f"CFB3_Ratio_{s}" for s in supplier]].sum()

            result_df.loc[d, [f"CFB12_Mixing_{p}" for p in param_list]] = [cfb12_mix[p][d]() for p in param_list]
            result_df.loc[d, [f"CFB3_Mixing_{p}" for p in param_list]] = [cfb3_mix[p][d]() for p in param_list]

            result_df.loc[d, 'CFB12_LimeFlow'] = cfb12_lime_flow[d]()
            result_df.loc[d, 'CFB12_AshFlow'] = cfb12_ash_flow[d]()
            result_df.loc[d, 'CFB12_CoalFlow'] = cfb12_coal_flow[d]()
            result_df.loc[d, 'CFB12_BoilerEff'] = cfb12_boiler_eff[d]()
            result_df.loc[d, 'CFB12_Use'] = cfb12_use_sum[d]()

            result_df.loc[d, 'CFB3_LimeFlow'] = cfb3_lime_flow[d]()
            result_df.loc[d, 'CFB3_AshFlow'] = cfb3_ash_flow[d]()
            result_df.loc[d, 'CFB3_CoalFlow'] = cfb3_coal_flow[d]()
            result_df.loc[d, 'CFB3_BoilerEff'] = cfb3_boiler_eff[d]()
            result_df.loc[d, 'CFB3_Use'] = cfb3_use_sum[d]()

            result_df.loc[d, [f"OP_CFB{n}" for n in [1, 2, 3]]] = df.loc[d, [f"OP_CFB{n}" for n in [1, 2, 3]]]

    return result_df

In [ ]:
def expand_schedule(df, show_solver_log=False):
    df = df.resample('D').bfill()
    
    # ______Constants_______
    date_rng = df.index.tolist()

    ratio_step = 10
    n_ratio_step = 100/ratio_step

    remain = {d: {s: 0.0 for s in supplier} for d in date_rng}
    remain_notzero = {d: {s: 0.0 for s in supplier} for d in date_rng[1:]}

    cfb12_ratio = {d: {s: 0.0 for s in supplier} for d in date_rng[1:]}
    cfb3_ratio = {d: {s: 0.0 for s in supplier} for d in date_rng[1:]}
    cfb12_use = {d: {s: 0.0 for s in supplier} for d in date_rng[1:]}
    cfb3_use = {d: {s: 0.0 for s in supplier} for d in date_rng[1:]}

    # ______Calculation______
    # set remain for day zero to be the same as first day of input df
    for s in supplier:
        remain[date_rng[0]][s] = df.loc[date_rng[0], f"Remain_{s}"]

    # set operation for each day
    for d in date_rng[1:]:
        df.loc[d, ['OP_CFB1', 'OP_CFB2', 'OP_CFB3']] = df_daily.loc[d, ['OP_CFB1', 'OP_CFB2', 'OP_CFB3']]

    # set coal incoming for each day
    for d in date_rng[1:]:
        df.loc[d, [f"In_{s}" for s in supplier]] = df_daily.loc[d, [f"In_{s}" for s in supplier]]

    # calculate integer ratio value for daily operation to achieved approximately weekly blending ratio
    cfb12_ratio_avg = [np.around(df.loc[date_rng[7*(i+1)], [f"CFB12_Ratio_{s}" for s in supplier]].values, 4) for i in range(Sim_Week_Inc_Step)]
    cfb3_ratio_avg = [np.around(df.loc[date_rng[7*(i+1)], [f"CFB3_Ratio_{s}" for s in supplier]].values, 4) for i in range(Sim_Week_Inc_Step)]

    for i, ratio, ratio_avg in zip(range(2), [cfb12_ratio, cfb3_ratio], [cfb12_ratio_avg, cfb3_ratio_avg]):
        # week 1 and week 2 have the same ratio
        if (ratio_avg[0] == ratio_avg[1]).all():
            n_supplier = [(s, ratio_avg[0][supplier.index(s)]) for s in supplier if ratio_avg[0][supplier.index(s)] > 0]

            if len(n_supplier) == 1:
                for d in date_rng[1:]:
                    ratio[d][n_supplier[0][0]] = n_supplier[0][1]
            else:
                ratio_set = np.array([x/14 for x in range(14)])
                q, mod = divmod(n_supplier[0][1], 1)
                idx = (np.abs(ratio_set - mod)).argmin()

                for i, d in enumerate(date_rng[1:]):
                    ratio[d][n_supplier[0][0]] = q + (0 if (13 - i) >= idx else 1)
                    ratio[d][n_supplier[1][0]] = n_ratio_step - ratio[d][n_supplier[0][0]]

        # week 1 and week 2 have diffrence ratio
        else:
            for i in range(Sim_Week_Inc_Step):
                n_supplier = [(s, ratio_avg[i][supplier.index(s)]) for s in supplier if ratio_avg[i][supplier.index(s)] > 0]
                
                if len(n_supplier) == 1:
                    for d in date_rng[1+7*i:8+7*(i)]:
                        ratio[d][n_supplier[0][0]] = n_supplier[0][1]
                else:
                    ratio_set = np.array([x/7 for x in range(7)])
                    q, mod = divmod(n_supplier[0][1], 1)
                    idx = (np.abs(ratio_set - mod)).argmin()

                    for i, d in enumerate(date_rng[1+7*i:8+7*(i)]):
                        ratio[d][n_supplier[0][0]] = q + (0 if (6 - i) >= idx else 1)
                        ratio[d][n_supplier[1][0]] = n_ratio_step - ratio[d][n_supplier[0][0]]
        
    # calculate coal mixing parameters
    cfb12_mix = {p: {d: sum(cfb12_ratio[d][s] * coal_df.loc[s, param_lookup_list[param_list.index(p)]] for s in supplier) 
                        / sum(cfb12_ratio[d][s] for s in supplier) for d in date_rng[1:]} for p in param_list}
    cfb3_mix = {p: {d: sum(cfb3_ratio[d][s] * coal_df.loc[s, param_lookup_list[param_list.index(p)]] for s in supplier)
                        / sum(cfb3_ratio[d][s] for s in supplier) for d in date_rng[1:]} for p in param_list}

    param_min = [17000, 0, 0 , 0, 0]
    param_max = [29000, 36, 10 , 1, 10]
    cfb12_mix_norm = {p: {d: norm(cfb12_mix[p][d]*(4.1868 if p == 'GCV' else 1), param_min[param_list.index(p)], param_max[param_list.index(p)]) for d in date_rng[1:]} for p in param_list[:5]}
    cfb3_mix_norm = {p: {d: norm(cfb3_mix[p][d]*(4.1868 if p == 'GCV' else 1), param_min[param_list.index(p)], param_max[param_list.index(p)]) for d in date_rng[1:]} for p in param_list[:5]}

    #Correction Tstack when Fe and Na over limit
    T_Fe12 = {d: (sigmoid(cfb12_mix['%Fe2O3'][d], 5, 1, 40, 11.5) + sigmoid(cfb12_mix['%Fe2O3'][d], 10, 1, 40, 14.5)) for d in date_rng[1:]}
    T_Fe3 = {d: (sigmoid(cfb3_mix['%Fe2O3'][d], 5, 1, 40, 11.5) + sigmoid(cfb3_mix['%Fe2O3'][d], 10, 1, 40, 14.5)) for d in date_rng[1:]}

    T_Na12 = {d: (sigmoid(cfb12_mix['%Na2O'][d], 5, 1, 40, 2.25) + sigmoid(cfb12_mix['%Na2O'][d], 10, 1, 40, 5)) for d in date_rng[1:]}
    T_Na3 = {d: (sigmoid(cfb3_mix['%Na2O'][d], 5, 1, 200, 2.25) + sigmoid(cfb3_mix['%Na2O'][d], 10, 1, 40, 5)) for d in date_rng[1:]}
    
    T12 = {d: norm(165 + T_Fe12[d] + T_Na12[d], min=120, max=185) for d in date_rng[1:]}
    T3 = {d: norm(165 + T_Fe3[d] + T_Na3[d], min=120, max=185) for d in date_rng[1:]}

    # calculate lime flow, boiler eff and coal consumption
    cfb12_lime_flow = {d: 1.098044 - 0.30009*Qro12_norm - 2.205101*cfb12_mix_norm['%S'][d] + 4.485632*(cfb12_mix_norm['%S'][d]**2) for d in date_rng[1:]}
    cfb12_lime_flow_norm = {d: norm(cfb12_lime_flow[d], min=0, max=3.5) for d in date_rng[1:]}
    cfb12_ash_flow = {d: -0.157 + 0.1464*Qro12_norm - 0.1778*cfb12_mix_norm['GCV'][d] + 0.7089*cfb12_lime_flow_norm[d] + 0.423*cfb12_mix_norm['%Ash'][d]
                    - 0.056*Excess_O2 + 0.1962*Pa_ratio for d in date_rng[1:]}
    cfb12_boiler_eff = {d: 98.2894 + 0.2862*Qro12_norm + 5.453*cfb12_mix_norm['GCV'][d] - 4.5807*cfb12_mix_norm['TM'][d]
                        - 17.9721*cfb12_mix_norm['%H'][d] - 0.4126*cfb12_mix_norm['%Ash'][d] - 2.7522*cfb12_lime_flow_norm[d]
                        - 2.9131*T12[d] - 2.6225*Excess_O2 for d in date_rng[1:]}

    cfb3_lime_flow = {d: 1.098044 - 0.30009*Qro3_norm - 2.205101*cfb3_mix_norm['%S'][d] + 4.485632*(cfb3_mix_norm['%S'][d]**2) for d in date_rng[1:]}
    cfb3_lime_flow_norm = {d: norm(cfb3_lime_flow[d], min=0, max=3.5) for d in date_rng[1:]}
    cfb3_ash_flow = {d: -0.157 + 0.1464*Qro3_norm - 0.1778*cfb3_mix_norm['GCV'][d] + 0.7089*cfb3_lime_flow_norm[d] + 0.423*cfb3_mix_norm['%Ash'][d]
                    - 0.056*Excess_O2 + 0.1962*Pa_ratio for d in date_rng[1:]}
    cfb3_boiler_eff = {d: 98.2894 + 0.2862*Qro3_norm + 5.453*cfb3_mix_norm['GCV'][d] - 4.5807*cfb3_mix_norm['TM'][d]
                        - 17.9721*cfb3_mix_norm['%H'][d] - 0.4126*cfb3_mix_norm['%Ash'][d] - 2.7522*cfb3_lime_flow_norm[d]
                        - 2.9131*T3[d] - 2.6225*Excess_O2 for d in date_rng[1:]}
                        
    # calculate coal use
    cfb12_coal_flow = {}
    cfb3_coal_flow = {}
    cfb12_use_sum = {}
    cfb3_use_sum = {}
    for d in date_rng[1:]:
        cfb12_coal_flow[d] = (Qro12*1000) / (cfb12_boiler_eff[d]/100) / (cfb12_mix['GCV'][d]*4.1868)
        cfb3_coal_flow[d] = (Qro3*1000) / (cfb3_boiler_eff[d]/100)  /(cfb3_mix['GCV'][d]*4.1868)

        cfb12_use_sum[d] = cfb12_coal_flow[d] * df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum() * (3600*24) / 1000
        cfb3_use_sum[d] = cfb3_coal_flow[d] * df.loc[d, 'OP_CFB3'] * (3600*24) / 1000

    for d, s in product(date_rng[1:], supplier):
        cfb12_use[d][s] = (cfb12_ratio[d][s] / n_ratio_step) * cfb12_use_sum[d]
        cfb3_use[d][s] = (cfb3_ratio[d][s] / n_ratio_step) * cfb3_use_sum[d]

    # calculate coal remain
    for d, s in product(date_rng[1:], supplier):
        remain[d][s] = remain[shift_list(date_rng, d, -1)][s] - cfb12_use[d][s] - cfb3_use[d][s] + df.loc[d, f"In_{s}"]

        # force remaining error to 0
        if abs(remain[d][s]) < 1000:
            remain[d][s] = 0

    # if remain > 0 then notzero = 1, else if remain == 0 then notzero = 0
    for d, s in product(date_rng[1:], supplier):
        remain_notzero[d][s] = 1 if remain[d][s] > 0 else 0

    # calculate cost
    coal_cost = {d: sum((cfb12_use[d][s] + cfb3_use[d][s]) * coal_df.loc[s, 'Coal Cost (THB/ton)'] for s in supplier) for d in date_rng[1:]}
    lime_cost = {d: (cfb12_lime_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum() + cfb3_lime_flow[d]*df.loc[d, 'OP_CFB3']) * (3600*24) / 1000 * lime_price for d in date_rng[1:]}
    flyash_cost = {d: (cfb12_ash_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum()*0.9 + cfb3_ash_flow[d]*df.loc[d, 'OP_CFB3']*0.9) * (3600*24) / 1000 * flyash_price  for d in date_rng[1:]}
    bottomash_cost = {d: (cfb12_ash_flow[d]*df.loc[d, ['OP_CFB1', 'OP_CFB2']].sum()*0.1 + cfb3_ash_flow[d]*df.loc[d, 'OP_CFB3']*0.1) * (3600*24) / 1000 * bottomash_price  for d in date_rng[1:]}
    fe2o3_cost = {d: cfb12_use_sum[d] * (sigmoid(cfb12_mix['%Fe2O3'][d], 355.34, 1, 40, 14.5)) + 
                cfb3_use_sum[d] * (sigmoid(cfb3_mix['%Fe2O3'][d], 355.34, 1, 40, 14.5)) for d in date_rng[1:]}
    na2o_cost = {d: cfb12_use_sum[d] * (sigmoid(cfb12_mix['%Na2O'][d], 355.34, 1, 40, 5)) + 
                cfb3_use_sum[d] * (sigmoid(cfb3_mix['%Na2O'][d], 355.34, 1, 40, 5)) for d in date_rng[1:]}
    remain_penalty = {d: sigmoid(sum(remain_notzero[d][s] for s in supplier), 1e8, 1, 100, 3.5) for d in date_rng[1:]}

    #_______Results________
    for d in date_rng[1:]:
        df.loc[d, 'Cost_Coal'] = coal_cost[d]
        df.loc[d, 'Cost_Lime'] = lime_cost[d]
        df.loc[d, 'Cost_FlyAsh'] = flyash_cost[d]
        df.loc[d, 'Cost_BottomAsh'] = bottomash_cost[d]
        df.loc[d, 'Cost_Fe2O3'] = fe2o3_cost[d]
        df.loc[d, 'Cost_Na2O'] = na2o_cost[d]
        df.loc[d, 'Cost_Total'] = df.loc[d, [f"Cost_{x}" for x in ['Coal', 'Lime', 'FlyAsh', 'BottomAsh', 'Fe2O3', 'Na2O']]].sum()
        df.loc[d, 'Penalty_Remain'] = remain_penalty[d]

        df.loc[d, [f"Use_{s}" for s in supplier]] = [cfb12_use[d][s] + cfb3_use[d][s] for s in supplier]
        df.loc[d, 'Use_Total'] = df.loc[d, [f"Use_{s}" for s in supplier]].sum()

        df.loc[d, 'In_Total'] = df.loc[d, [f"In_{s}" for s in supplier]].sum()

        df.loc[d, [f"Remain_{s}" for s in supplier]] = [remain[d][s] for s in supplier]
        df.loc[d, 'Remain_Total'] = df.loc[d, [f"Remain_{s}" for s in supplier]].sum()

        df.loc[d, [f"NotZero_{s}" for s in supplier]] = [remain_notzero[d][s] for s in supplier]
        df.loc[d, 'NotZero_Total'] = df.loc[d, [f"NotZero_{s}" for s in supplier]].sum()

        df.loc[d, [f"CFB12_Ratio_{s}" for s in supplier]] = [cfb12_ratio[d][s] for s in supplier]
        df.loc[d, 'CFB12_Ratio_Total'] = df.loc[d, [f"CFB12_Ratio_{s}" for s in supplier]].sum()
        df.loc[d, [f"CFB3_Ratio_{s}" for s in supplier]] = [cfb3_ratio[d][s] for s in supplier]
        df.loc[d, 'CFB3_Ratio_Total'] = df.loc[d, [f"CFB3_Ratio_{s}" for s in supplier]].sum()

        df.loc[d, [f"CFB12_Mixing_{p}" for p in param_list]] = [cfb12_mix[p][d] for p in param_list]
        df.loc[d, [f"CFB3_Mixing_{p}" for p in param_list]] = [cfb3_mix[p][d] for p in param_list]

        df.loc[d, 'CFB12_LimeFlow'] = cfb12_lime_flow[d]
        df.loc[d, 'CFB12_AshFlow'] = cfb12_ash_flow[d]
        df.loc[d, 'CFB12_CoalFlow'] = cfb12_coal_flow[d]
        df.loc[d, 'CFB12_BoilerEff'] = cfb12_boiler_eff[d]
        df.loc[d, 'CFB12_Use'] = cfb12_use_sum[d]

        df.loc[d, 'CFB3_LimeFlow'] = cfb3_lime_flow[d]
        df.loc[d, 'CFB3_AshFlow'] = cfb3_ash_flow[d]
        df.loc[d, 'CFB3_CoalFlow'] = cfb3_coal_flow[d]
        df.loc[d, 'CFB3_BoilerEff'] = cfb3_boiler_eff[d]
        df.loc[d, 'CFB3_Use'] = cfb3_use_sum[d]

    return df

In [ ]:
# Simulation
n_sim = math.ceil((float(Sim_Week_Total) - float(Sim_Week_Duration)) / float(Sim_Week_Inc_Step) + 1)
for i in tqdm(range(n_sim + 1)):
    # print(f"Iter {i + 1}/{n_sim + 1}")

    if i <= n_sim:
        # Extract part of df for only simalation duration and simulate
        df_week = df.iloc[Sim_Week_Inc_Step * i:Sim_Week_Inc_Step * i + Sim_Week_Duration + 1, :]
        # print(df_week)

        result_week = simulation(df_week, show_solver_log=False)
        print(result_week)

        result_daily = expand_schedule(result_week.iloc[0:Sim_Week_Inc_Step + 1], True)
        # print(result_daily)

        # Update remaining coal to main df
        df.loc[result_daily.index.tolist()[-1], [f"Remain_{s}" for s in supplier]] = result_daily.iloc[-1][[f"Remain_{s}" for s in supplier]].values

        # Save result to result_df
        if i == 0:
            result_df_daily = result_daily
            result_df_week = result_week.iloc[0:Sim_Week_Inc_Step + 1]
        else:
            result_df_daily = result_df_daily.append(result_daily.tail(Sim_Week_Inc_Step * 7))
            result_df_week = result_df_week.append(result_week.iloc[1:Sim_Week_Inc_Step + 1])

    else:
        result_daily = expand_schedule(result_week.iloc[-(Sim_Week_Inc_Step + 1):], True)
        result_df_daily = result_df_daily.append(result_daily.tail(Sim_Week_Inc_Step * 7))
        result_df_week = result_df_week.append(result_week.tail(Sim_Week_Inc_Step))
        
# Export result data
from google.colab import files
result_df_daily.to_excel('result.xlsx', sheet_name='Result')
files.download('result.xlsx')
result_df_daily.columns = pd.MultiIndex.from_tuples([(col[:col.rfind('_')], col[col.rfind('_') + 1:]) for col in result_df_daily.columns])

print('\nResults:')
result_df_week.columns = pd.MultiIndex.from_tuples([(col[:col.rfind('_')], col[col.rfind('_') + 1:]) for col in result_df_week.columns])
result_df_week

  0%|          | 0/26 [00:00<?, ?it/s]

               Cost_Coal  Cost_Lime  Cost_FlyAsh  Cost_BottomAsh  Cost_Fe2O3  Cost_Na2O    Cost_Total  Penalty_Remain  Use_Banpu  Use_Logplus  Use_Mac  Use_Tiger  Use_AVRA  Use_Spot1  Use_Spot2  Use_Spot3  Use_Total  In_Banpu  In_Logplus  In_Mac  In_Tiger  In_AVRA  In_Spot1  In_Spot2  In_Spot3  In_Total  Remain_Banpu  Remain_Logplus  Remain_Mac  Remain_Tiger  Remain_AVRA  Remain_Spot1  Remain_Spot2  Remain_Spot3  Remain_Total  NotZero_Banpu  NotZero_Logplus  NotZero_Mac  NotZero_Tiger  NotZero_AVRA  NotZero_Spot1  NotZero_Spot2  NotZero_Spot3  NotZero_Total  CFB12_Ratio_Banpu  CFB12_Ratio_Logplus  CFB12_Ratio_Mac  CFB12_Ratio_Tiger  CFB12_Ratio_AVRA  CFB12_Ratio_Spot1  CFB12_Ratio_Spot2  CFB12_Ratio_Spot3  CFB12_Ratio_Total  CFB3_Ratio_Banpu  CFB3_Ratio_Logplus  CFB3_Ratio_Mac  CFB3_Ratio_Tiger  CFB3_Ratio_AVRA  CFB3_Ratio_Spot1  CFB3_Ratio_Spot2  CFB3_Ratio_Spot3  CFB3_Ratio_Total  CFB12_Mixing_GCV  CFB12_Mixing_TM  CFB12_Mixing_%H  CFB12_Mixing_%S  CFB12_Mixing_%Ash  CFB12_Mixing_%Fe

  4%|▍         | 1/26 [00:32<13:34, 32.56s/it]

               Cost_Coal  Cost_Lime  Cost_FlyAsh  Cost_BottomAsh  Cost_Fe2O3  Cost_Na2O    Cost_Total  Penalty_Remain  Use_Banpu  Use_Logplus  Use_Mac  Use_Tiger  Use_AVRA  Use_Spot1  Use_Spot2  Use_Spot3  Use_Total  In_Banpu  In_Logplus  In_Mac  In_Tiger  In_AVRA  In_Spot1  In_Spot2  In_Spot3  In_Total  Remain_Banpu  Remain_Logplus  Remain_Mac  Remain_Tiger  Remain_AVRA  Remain_Spot1  Remain_Spot2  Remain_Spot3  Remain_Total  NotZero_Banpu  NotZero_Logplus  NotZero_Mac  NotZero_Tiger  NotZero_AVRA  NotZero_Spot1  NotZero_Spot2  NotZero_Spot3  NotZero_Total  CFB12_Ratio_Banpu  CFB12_Ratio_Logplus  CFB12_Ratio_Mac  CFB12_Ratio_Tiger  CFB12_Ratio_AVRA  CFB12_Ratio_Spot1  CFB12_Ratio_Spot2  CFB12_Ratio_Spot3  CFB12_Ratio_Total  CFB3_Ratio_Banpu  CFB3_Ratio_Logplus  CFB3_Ratio_Mac  CFB3_Ratio_Tiger  CFB3_Ratio_AVRA  CFB3_Ratio_Spot1  CFB3_Ratio_Spot2  CFB3_Ratio_Spot3  CFB3_Ratio_Total  CFB12_Mixing_GCV  CFB12_Mixing_TM  CFB12_Mixing_%H  CFB12_Mixing_%S  CFB12_Mixing_%Ash  CFB12_Mixing_%Fe

  8%|▊         | 2/26 [13:42<3:11:06, 477.78s/it]

...

100%|██████████| 26/26 [45:27<00:00, 104.90s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Results:


Cost                                                                    Penalty       Use                                                                                       In                                                                               Remain                                                                                  NotZero                                                 CFB12_Ratio                                                 CFB3_Ratio                                                 CFB12_Mixing                                   CFB3_Mixing                                      CFB12                                          CFB3                                       OP          
                    Coal       Lime     FlyAsh BottomAsh  Fe2O3   Na2O         Total         Remain     Banpu   Logplus       Mac     Tiger     AVRA     Spot1     Spot2     Spot3     Total     Banpu   Logplus       Mac     Tiger AVRA     Spot1     Spot2     Spot3     Total     Banpu    Logplus       Mac     Tiger     AVRA     Spot1     Spot2     Spot3      Total   Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total       Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total      Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total          GCV    TM   %H   %S %Ash %Fe2O3 %Na2O         GCV    TM   %H   %S %Ash %Fe2O3 %Na2O LimeFlow AshFlow CoalFlow BoilerEff       Use LimeFlow AshFlow CoalFlow BoilerEff      Use CFB1 CFB2 CFB3
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
2019-12-31           nan        nan        nan       nan    nan    nan           nan            nan       nan       nan       nan       nan      nan       nan       nan       nan       nan      0.00      0.00      0.00      0.00 0.00      0.00      0.00      0.00       nan      0.00  69,336.07      0.00 19,000.00 1,602.00      0.00      0.00      0.00        nan     nan     nan  nan   nan  nan   nan   nan   nan   nan         nan     nan  nan   nan  nan   nan   nan   nan   nan        nan     nan  nan   nan  nan   nan   nan   nan   nan          nan   nan  nan  nan  nan    nan   nan         nan   nan  nan  nan  nan    nan   nan      nan     nan      nan       nan       nan      nan     nan      nan       nan      nan 0.00 0.00 0.00
2020-01-07 56,530,069.36 788,540.86 109,314.07 36,824.27   0.00 279.90 57,465,028.45           0.00      0.00  8,390.64      0.00 16,568.12     0.00      0.00      0.00      0.00 24,958.76      0.00      0.00      0.00 33,862.12 0.00      0.00      0.00      0.00 33,862.12      0.00  60,945.43      0.00 36,294.00 1,602.00      0.00      0.00      0.00  98,841.42    0.00    1.00 0.00  1.00 1.00  0.00  0.00  0.00  3.00        0.00    3.36 0.00  6.64 0.00  0.00  0.00  0.00 10.00       0.00    3.36 0.00  6.64 0.00  0.00  0.00  0.00 10.00     5,578.83 19.58 4.23 0.28 4.40  10.01  4.74    5,578.76 19.58 4.23 0.28 4.39  10.01  4.74     0.63    0.27    13.96     87.71 16,701.08     0.64    0.27    13.77     87.69 8,257.68 6.88 6.96 6.94
2020-01-14 54,102,019.17 845,932.60 117,846.87 39,698.68   0.00 185.08 55,105,682.41           0.00      0.00 12,082.82      0.00 11,082.11 1,602.00      0.00      0.00      0.00 24,766.93 54,004.86      0.00      0.00      0.00 0.00      0.00      0.00      0.00 54,004.86 54,004.86  48,862.61      0.00 25,211.88     0.00  

In [ ]:
print(result_df_daily.describe().append(pd.Series(result_df_daily.sum(axis=0), name='sum')))
result_df_daily

                  Cost                                                                                              Penalty        Use                                                                                             In                                                                                         Remain                                                                                                    NotZero                                                            CFB12_Ratio                                                             CFB3_Ratio                                                             CFB12_Mixing                                                    CFB3_Mixing                                                      CFB12                                           CFB3                                           OP              
                  Coal          Lime       FlyAsh    BottomAsh         Fe2O3         Na2O            Total           Remain  

Cost                                                             Penalty   Use                                                              In                                                            Remain                                                                    NotZero                                                 CFB12_Ratio                                                 CFB3_Ratio                                                 CFB12_Mixing                                   CFB3_Mixing                                      CFB12                                         CFB3                                       OP          
                   Coal       Lime    FlyAsh BottomAsh Fe2O3      Na2O        Total  Remain Banpu  Logplus  Mac    Tiger AVRA Spot1 Spot2    Spot3    Total Banpu Logplus  Mac     Tiger AVRA Spot1 Spot2 Spot3     Total     Banpu   Logplus  Mac     Tiger     AVRA Spot1 Spot2     Spot3      Total   Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total       Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total      Banpu Logplus  Mac Tiger AVRA Spot1 Spot2 Spot3 Total          GCV    TM   %H   %S %Ash %Fe2O3 %Na2O         GCV    TM   %H   %S %Ash %Fe2O3 %Na2O LimeFlow AshFlow CoalFlow BoilerEff      Use LimeFlow AshFlow CoalFlow BoilerEff      Use CFB1 CFB2 CFB3
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
2019-12-31          nan        nan       nan       nan   nan       nan          nan     nan   nan      nan  nan      nan  nan   nan   nan      nan      nan  0.00    0.00 0.00      0.00 0.00  0.00  0.00  0.00       nan      0.00 69,336.07 0.00 19,000.00 1,602.00  0.00  0.00      0.00        nan     nan     nan  nan   nan  nan   nan   nan   nan   nan         nan     nan  nan   nan  nan   nan   nan   nan   nan        nan     nan  nan   nan  nan   nan   nan   nan   nan          nan   nan  nan  nan  nan    nan   nan         nan   nan  nan  nan  nan    nan   nan      nan     nan      nan       nan      nan      nan     nan      nan       nan      nan 0.00 0.00 0.00
2020-01-01 8,192,632.97 113,041.76 15,584.18  5,249.79  0.00 27,061.17 8,353,569.88    0.00  0.00 1,080.50 0.00 2,521.17 0.00  0.00  0.00     0.00 3,601.66  0.00    0.00 0.00 33,862.12 0.00  0.00  0.00  0.00 33,862.12      0.00 68,255.57 0.00 50,340.95 1,602.00  0.00  0.00      0.00 120,198.52    0.00    1.00 0.00  1.00 1.00  0.00  0.00  0.00  3.00        0.00    3.00 0.00  7.00 0.00  0.00  0.00  0.00 10.00       0.00    3.00 0.00  7.00 0.00  0.00  0.00  0.00 10.00     5,566.09 19.68 4.22 0.27 4.33  10.00  4.90    5,566.09 19.68 4.22 0.27 4.33  10.00  4.90     0.63    0.27    14.00     87.67 2,413.87     0.64    0.27    13.81     87.66 1,187.79 1.00 1.00 1.00
2020-01-02 8,169,865.17 112,726.12 15,540.84  5,235.19  0.00 26,985.97 8,330,353.28    0.00  0.00 1,077.50 0.00 2,514.16 0.00  0.00  0.00     0.00 3,591.66  0.00    0.00 0.00      0.00 0.00  0.00  0.00  0.00      0.00      0.00 67,178.07 0.00 47,826.79 1,602.00  0.00  0.00      0.00 116,606.86    0.00    1.00 0.00  1.00 1.00  0.00  0.00  0.00  3.00        0.00    3.00 0.00  7.00 0.00  0.00  0.00  0.00 10.00       0.00    3.00 0.00  7.00 0.00  0.00  0.00  0.00 10.00     5,566.09 19.68 4.22 0.27 4.33  10.00  4.90    5,566.09 19.68 4.22 0.27 4.33  10.00  4.90     0.63    0.27    14.00     87.67 2,408.84     0.64    0.27    13.81     87.66 1,182.82 0.99 1.00 0.99
2020-01-03 